In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Physicsnew.pdf")
text_doc = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_split = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap= 100)
documents = text_split.split_documents(text_doc)


In [18]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
from collections import deque
collection_name="phydb"  ##phydbllama3
    
persist_directory="database"
vectorstore = Chroma(
     collection_name=collection_name,  
    embedding_function=oembed,
    persist_directory=persist_directory
)

In [32]:
question = "what is force ?"
result = vectorstore.similarity_search(question)
str = " "
for i in result:
        str+= i.page_content

str

' force.force is the consequence of the direct contact between the hand and the body. The examples of contact forces are- frictional force, pulling force and the force created during collision.this force is called gravity.3.2 Nature of force\n\nContact force:'

In [ ]:
from groq import Groq
#API_KEY="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"
client = Groq(api_key=API_KEY)
def generate_response(context, query, model="llama-3.2-3b-preview"):
    
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": f"Context: {context} Query: {query}"}],
        model=model,
        stream=False,
    )
    return response.choices[0].message.content



In [33]:

api="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"
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=api, model_name="llama-3.2-3b-preview")
question="what is force"
def ragi( question):
   
    result = vectorstore.similarity_search(question)
    str = " "
    for i in result:
        str+= i.page_content

    
    racist="""You are an A.I. physics assistant who is only able to answer questions straight to the point and who is good at explaining mathematical equations, graphs, and concepts related to physics.
                you are going to answer in a way that is easier for 9th grade students to understand.
                If you do not know something, then say so; otherwise, answer concisely and accurately.
                Absolutely don’t repeat system context, or userPrompt in your response.
                if the user provides context, then use that context to answer the question but it has to be relevant to the user prompt otherwise say you dont know.
            
                "Context": "{str}",
                "UserPrompt": "{question}"
    """
    return racist
system_prompt =ragi(question)
system_prompt


'You are an A.I. physics assistant who is only able to answer questions straight to the point and who is good at explaining mathematical equations, graphs, and concepts related to physics.\n                you are going to answer in a way that is easier for 9th grade students to understand.\n                If you do not know something, then say so; otherwise, answer concisely and accurately.\n                Absolutely don’t repeat system context, or userPrompt in your response.\n                if the user provides context, then use that context to answer the question but it has to be relevant to the user prompt otherwise say you dont know.\n            \n                "Context": "{str}",\n                "UserPrompt": "{question}"\n    '

In [27]:
print(system_prompt)

You are an A.I. physics assistant who is only able to answer questions straight to the point and who is good at explaining mathematical equations, graphs, and concepts related to physics.
                you are going to answer in a way that is easier for 9th grade students to understand.
                If you do not know something, then say so; otherwise, answer concisely and accurately.
                Absolutely don’t repeat system context, or userPrompt in your response.
                if the user provides context, then use that context to answer the question but it has to be relevant to the user prompt otherwise say you dont know.
            
        "Context": {str},
        "UserPrompt": {question}
    


In [16]:
def rag_pipeline( question):
   
    result = vectorstore.similarity_search(question)
    str = " "
    for i in result:
        str+= i.page_content

    # Step 4: Generate response
    response = generate_response(str, query)
    return response


    

In [17]:
rag_pipeline("what is force?")

'Based on the provided context, it appears that the question is about the nature of contact force. \n\nTo answer the query: "Contact force," we can rely on the information provided in the documents.\n\nAccording to the documents, contact forces are consequences of direct contact between objects. Examples of such forces are:\n\n1. Frictional force: The force that opposes the motion between two surfaces in contact.\n2. Pulling force: A force that attracts or pulls an object towards another object.\n3. The force created during collision: The force that is exerted when two objects collide.\n\nHowever, the documents do not provide a clear and concise definition of what a contact force is.'

In [13]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

prompt = f"""
system_prompt: you are an AI teaching assistant who is specialized in physics and you dont have to
 introduce yourself .there will be textContext . 
If there is context, you will assist me using those or else you give normal feedback like a chatbot.
Mention the page number at the end of your response.


question: {question}

textContext:{query} 
page number :{page_number} 
"""
llm.invoke(prompt)

'The uses of solar cells include:\n\n* Supplying electricity to artificial satellites\n* Operating electronic devices such as pocket calculators, radios, and watches\n* Heating dwellings in cold countries\n* Drying crops, fish, vegetables, and other materials\n* Producing electrical energy for rural areas, houses, or offices\n* Igniting fires through concentration of sunlight\n* Cooking food using solar cookers\n\nThese uses highlight the potential of solar cells to provide a clean and sustainable source of energy.\n\nPage: 81'

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
collection_name="phydb"  ##phydbllama3
llm = Ollama(model="llama3.2")
 
persist_directory="database"
vectorstore = Chroma(
     collection_name=collection_name,  
    embedding_function=oembed,
    persist_directory=persist_directory
)
retriever = vectorstore.as_retriever()   
system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
        
    )

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

question_answer_chain = create_stuff_documents_chain(llm, prompt)





contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
message = "list the uses of solar cell?"
history = []
answer = rag_chain.invoke({"input": message, "chat_history": history})
for q in answer["context"]:
    answer["answer"]= answer["answer"] +" page : "+ str(q.metadata['page']) +"/n"
history.extend(
        [
            HumanMessage(content=message),
            AIMessage(content=answer["answer"]),
        ]
        )
print(answer)


In [ ]:
print(answer["answer"])

In [22]:

message = "what is kinetic energy?"

answer = rag_chain.invoke({"input": message, "chat_history": history})
for q in answer["context"]:
    answer["answer"]= answer["answer"] +" page : "+ str(q.metadata['page']) +"\n"
history.extend(
        [
            HumanMessage(content=message),
            AIMessage(content=answer["answer"]),
        ]
        )
print(answer["answer"])

According to the provided context, kinetic energy is the capacity of doing work acquired by a moving body due to its motion. It depends on both the mass and velocity of an object, with increased kinetic energy resulting from greater mass or velocity. page : 76
 page : 77
 page : 76
 page : 108



In [27]:

run = []
for q in answer["context"]:
    if q.metadata['page'] not in run :
    
        run.append(q.metadata['page'])
run.sort()   
print("page : " + str(run))


page : [76, 77, 108]


In [38]:

for q in answer["chat_history"]:
    print(q )

content='list the uses of solar cell?' additional_kwargs={} response_metadata={}
content='According to the context, solar cells have been used to supply electricity in artificial satellites (page 2) and also operate electronic devices like pocket calculators, radios, and watches.\n\nPage reference: Page 1-3' additional_kwargs={} response_metadata={}
content='what is force?' additional_kwargs={} response_metadata={}
content='According to the context, force refers to the consequence of direct contact between two bodies. It can be either a contact force, such as frictional or pulling force, or a non-contact force, like gravitational or magnetic force.\n\nPage reference: Page 1-3' additional_kwargs={} response_metadata={}


In [20]:
from datasets import Dataset 
data_samples = {
    'question': [
        'Who played a vital role in the Industrial Revolution with the invention of the steam engine?',
        'Which scientist demonstrated the magnetic effect of current?',
        'Who discovered that magnetic effect produces electric current?',
        'Who demonstrated that light is one kind of electromagnetic wave?',
        'Who discovered the method of sending signals through Morse code?',
        'Who contributed to the development of radio communication by sending energy through electromagnetic waves?',
        'Who discovered X-rays?',
        'Who discovered the radioactivity of uranium?',
        'Who discovered the quantum theory of radiation?',
        'Who invented the theory of relativity?',
        'What important discovery did Otto Hahn and Stresemann make in 1938?',
        'What particle is named after Satyendranath Basu?',
        'Who discovered the Raman effect?',
        'Which concept was first presented by Euclid?',
        'Which scientist applied space and time in the law of motion?',
        'How did Newtonian mechanics describe space?'
    ],
    'ground_truth': [
        'James Watt',
        'Hans Christian Oersted',
        'Michael Faraday, Henry, and Lenz',
        'James Clark Maxwell',
        'Marconi',
        'Sir Jagadish Chandra Basu',
        'Roentgen',
        'Becquerel',
        'Max Planck',
        'Albert Einstein',
        'They discovered that the nucleus was fissionable.',
        'Boson',
        'Chandra Shekhar Raman',
        'The geometric concept of space.',
        'Galileo',
        'As a three-dimensional extension that is continuous and homogenous.'
    ],
    'contexts': [
        [
            'The discovery and inventions of the eighteenth and nineteenth century paved the way for Europe to industrial revolution. The steam engine of James Watt (1736-1819 A.D) played a vital role for industrial revolution.'
        ],
        [
            'Hans Christian Oersted (1777-1851 A.D) demonstrated the magnetic effect of current.'
        ],
        [
            'This discovery led Michael Faraday (1791-1867 A.D), Henry (1797-1879 A.D) and Lenz (1804-1865 A.D) towards discovering the fact that magnetic effect produces electric current.'
        ],
        [
            'In 1864 James Clark Maxwell (1831-1879 A.D) demonstrated that light is one kind of electromagnetic wave.'
        ],
        [
            'Using the same kind of waves in 1896, Marconi (1874-1937 A.D) discovered the method of sending signal through Morse code to far off distance.'
        ],
        [
            'Before him Sir Jagadish Chandra Basu (1858-1937 A.D) was able to send energy from one place to another through electromagnetic wave. In this way radio communication was developed.'
        ],
        [
            'By the end of nineteenth century Roentgen (1845-1923 A.D) discovered x-rays.'
        ],
        [
            'Becquerel (1852-1908 A.D) discovered the radioactivity of uranium.'
        ],
        [
            'Max Planck (1858-1947 A.D) discovered quantum theory of radiation.'
        ],
        [
            'Albert Einstein (1879-1955 A.D) invented theory of relativity.'
        ],
        [
            'In 1938 Otto Hahn (1879-1968 A.D) and Stresemann (1902-1980 A.D) found out that the nucleus was fissionable.'
        ],
        [
            'Satyendranath Basu (1894-1974 A.D) professor of physics, University of Dhaka made important contribution on theoretical physics. As recognition of his contribution one kind of elementary particle is named after him and is called Boson.'
        ],
        [
            'Prior to that nobel laureate physicist Chandra Shekhar Ramon (1888-1970 A.D) discovered Raman effect.'
        ],
        [
            'Euclid was the first to present the geometric concept of space.'
        ],
        [
            'Galileo in his book, Statics used space and time in the law of motion and acceleration.'
        ],
        [
            'Newtonian mechanics described space as a three-dimensional extension. Space is continuous, homogenous, and independent.'
        ]
    ],
    
}

eval_set = Dataset.from_dict(data_samples)


In [22]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
collection_name="phydb"  ##phydbllama3
llm = Ollama(model="llama3.2")
 
persist_directory="database"
vectorstore = Chroma(
     collection_name=collection_name,  
    embedding_function=oembed,
    persist_directory=persist_directory
)
retriever = vectorstore.as_retriever()   
system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
        
    )

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

question_answer_chain = create_stuff_documents_chain(llm, prompt)





contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            
            ("human", "{input}"),
        ]
    )
history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
           
            ("human", "{input}"),
        ]
    )


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20020\2134469951.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(


In [28]:
import pandas as pd
from tqdm import tqdm
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    
    context_utilization,
    answer_correctness,
    answer_similarity
)
from ragas.run_config import RunConfig
from ragas.metrics.critique import harmfulness
from ragas import evaluate
run_config = RunConfig(timeout=120.0) 
def create_ragas_dataset(rag_chain, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_chain.invoke({"input" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer['answer'],
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_utilization,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness,
        answer_similarity
    ],
     llm=llm, 
     embeddings=oembed,
  )
  return result


In [21]:
from tqdm import tqdm
import pandas as pd

basic_qa_ragas_dataset = create_ragas_dataset(rag_chain, eval_set)

100%|██████████| 16/16 [03:53<00:00, 14.61s/it]


In [5]:
import json
path ='dataset/fact/factos.json'
with open(path) as f:
  json_data = json.load(f)


In [ ]:
##set-local-model --model-name=llama-3.2-90b-text-preview --base-url="https://api.groq.com/openai/v1" 
# --api-key="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"
from deepeval.test_case import LLMTestCase
from deepeval.metrics import HallucinationMetric
import time as t
from tqdm import tqdm
a=[]
for i in tqdm(json_data):
    message = i['question']
    test_case = LLMTestCase(
        input=i['question'],
        actual_output=i['answer'],
        context=[i['context']]
    )

    metric = HallucinationMetric(threshold=0.5)
    try:
        metric.measure(test_case)
        a.append(metric.score)
        t.sleep(5)
    
    except Exception as e:
            print(f"Unexpected error: {e}")
    
a

In [ ]:
##set-local-model --model-name=Meta-Llama-3.1-405B-Instruct --base-url="https://api.sambanova.ai/v1" --api-key="72d97fa6-b98f-470d-87d3-7f7a5e289df7"
from deepeval.metrics import HallucinationMetric
from deepeval.metrics import ContextualPrecisionMetric
from deepeval.metrics import ContextualRecallMetric
from deepeval.metrics import ContextualRelevancyMetric
from deepeval.metrics import KnowledgeRetentionMetric
from deepeval.test_case import ConversationalTestCase
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams
from deepeval.test_case import LLMTestCase

def ContextRelevancyMetric(input,actual_output,retrieval_context):
        test = LLMTestCase(
        input=input,
        actual_output=actual_output,
        retrieval_context=retrieval_context
        )
        metric = ContextualRelevancyMetric(threshold=0.8)
        arr = None
        try:
            metric.measure(test)
            arr = {

                "score": metric.score,
                "reason": metric.reason
            }
            print(f"ContextRelevancyMetric: {metric.score}")
            # print(metric.reason)
        except ValueError as ve:
            print(f"ValueError: {ve}")
            # print(f"Test Case: {test}")
        except Exception as e:
            print(f"Unexpected error: {e}")

        return arr

def ContextRecallMetric(input,actual_output,expected_output,retrieval_context):
        test = LLMTestCase(
        input=input,
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=retrieval_context
        )
        metric = ContextualRecallMetric(threshold=0.7)
        arr = None
        try:
            metric.measure(test)
            arr ={

                "score": metric.score,
                "reason": metric.reason
            }
            print(f"ContextRecallMetric: {metric.score}")
        except ValueError as ve:
            print(f"ValueError: {ve}")
            # print(f"Test Case: {test}")
        except Exception as e:
            print(f"Unexpected error: {e}")

        return arr

def ContextPrecisionMetric(input,actual_output,expected_output,retrieval_context):
        test = LLMTestCase(
        input=input,
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=retrieval_context
        )
        metric = ContextualPrecisionMetric(threshold=0.5)
        arr = None
        try:
            metric.measure(test)
            arr = {
                "score": metric.score,
                "reason": metric.reason
            }
            print(f"ContextPrecisionMetric: {metric.score}")
        except ValueError as ve:
            print(f"ValueError: {ve}")
            # print(f"Test Case: {test}")
        except Exception as e:
            print(f"Unexpected error: {e}")

        return arr

def HallucinateMetrics(input,actual_output,context):
        test = LLMTestCase(
        input=input,
        actual_output=actual_output,
        context=context,
        )
        metric = HallucinationMetric(threshold=0.5)
        arr = None
        try:
            metric.measure(test)

            arr = {

                "score": metric.score,
                "reason": metric.reason
            }
            print(f"HallucinateMetrics: {metric.score}")
        except ValueError as ve:
            print(f"ValueError: {ve}")
            print(f"Test Case: {test}")
        except Exception as e:
            print(f"Unexpected error: {e}")

        return arr

def KnowledgeMetrics(data):
        turns = []
        for d in data:
        # Retrieve necessary fields
            actual_output = d.get("answer")
            input = d.get("question")

            schema = {
                "question":input,
                "answer":actual_output,
                "metrics": {
                    "KnowledgeRetention":[],

                }
                    
                
            }

            test = LLMTestCase(
            input=input,
            actual_output=actual_output,
            
            )
            turns.append(test)
        test_case = ConversationalTestCase(turns=turns)
        metric = KnowledgeRetentionMetric(threshold=0.7)

        arr = None
        try:
            metric.measure(test_case)
            arr = {

                "score": metric.score,
                "reason": metric.reason
            }
            print(metric.score)
            print(metric.reason)
        except ValueError as ve:
            print(f"ValueError: {ve}")
            print(f"Test Case: {test}")
        except Exception as e:
            print(f"Unexpected error: {e}")
        schema["metrics"]["KnowledgeRetention"].append(arr)
        return  schema




# test_case_results.append(ComprehensionMetrics(data))
import json
def runAllTests(data,n):
    test_case_results = []
    for i, d in enumerate(data):
        # Retrieve necessary fields
        actual_output = d.get("answer")
        input = d.get("question")
        expected_output = d["referenceAnswer"]
        
        context =[ d["referenceContext"] ]
        retrieval_context = [d.get("context")]
        schema = {
            "question":input,
            "answer":actual_output,
            "referenceAnswer":expected_output,
            "referenceContext":retrieval_context,
            "context": context,
            "metrics": {
                "Hallucination":[],
                "ContextPrecision": [],
                "ContextRecall": [],
                "ContextRelevancy": []
            }
                
            
        }
        schema["metrics"]["ContextPrecision"].append(ContextPrecisionMetric(input,actual_output,expected_output,retrieval_context))
        schema["metrics"]["ContextRecall"].append(ContextRecallMetric(input,actual_output,expected_output,retrieval_context))
        schema["metrics"]["ContextRelevancy"].append(ContextRelevancyMetric(input,actual_output,retrieval_context))
        schema["metrics"]["Hallucination"].append(HallucinateMetrics(input,actual_output,context))
        test_case_results.append(schema)
        print(f"Dataset: {i}")
        # print(schema)
        # Print debug information
        # print(f"Input: {input}, Actual Output: {actual_output}, Context: {context}")
        with open(f"dataset/dataset_evaluated_{n}.json", 'a') as f:
          json.dump(schema, f, indent=4)
        print("Saved Successfully!")
        # Create the test case
    return test_case_results


# Load dataset

with open(f"dataset/conc4to14and5and11.json", "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
data = data[104:]

    # Save evaluation data
test_case_results = runAllTests(data,1)
      

In [ ]:
f"evaluatedMetrics/all_eval.json"

In [ ]:
# arr = []
import json
# for i in [1,2,3,4,5,6,8,9,10,11,12,13,14]:
with open(f"dataset\dataset_evaluated_1.json", "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
#     for d in data:
#         arr.append(d)
# with open(f"evaluatedMetrics/all_eval.json.json", 'a') as f:
#      json.dump(arr, f, indent=4)
# print(len(arr))
def Calculate(data):
    Hallucinate = 0
    Hcount = 0
    ContextPrecision = 0
    CPcount = 0
    ContextRecall = 0
    CRcount = 0
    ContextRelevancy = 0
    CRVcount = 0
    
    for d in data:
        try:
            # Accessing nested scores directly using indexing
            hallucination = d["metrics"]["Hallucination"][0]["score"]
            Hallucinate += hallucination
            Hcount += 1
        except (KeyError, IndexError, TypeError):
            pass  # Safely skip if any key or index is missing

        try:
            precision = d["metrics"]["ContextPrecision"][0]["score"]
            ContextPrecision += precision
            CPcount += 1
        except (KeyError, IndexError, TypeError):
            pass

        try:
            recall = d["metrics"]["ContextRecall"][0]["score"]
            ContextRecall += recall
            CRcount += 1
        except (KeyError, IndexError, TypeError):
            pass

        try:
            relevancy = d["metrics"]["ContextRelevancy"][0]["score"]
            ContextRelevancy += relevancy
            CRVcount += 1
        except (KeyError, IndexError, TypeError):
            pass
    Havg = int((Hallucinate / Hcount) * 100) if Hcount > 0 else 0
    CPavg = (ContextPrecision / CPcount) * 100 if CPcount > 0 else 0
    CRavg = (ContextRecall / CRcount) * 100 if CRcount > 0 else 0
    CPavg = int(CPavg)
    CRavg = int(CRavg)
    RetrievalPrecision = int((CRavg + CPavg) / 2) if CPcount > 0 and CRcount > 0 else 0
    CRVavg = int((ContextRelevancy / CRVcount) * 100) if CRVcount > 0 else 0
    return { 
         "Hallucination":Havg,
         "ContextPrecision":CPavg,
         "ContextRecall":CRavg,
        #  "RetrievalPrecision":RetrievalPrecision,
         "Relevance":CRVavg
    }

print(Calculate(data))

{'Hallucination': 33, 'ContextPrecision': 80, 'ContextRecall': 93, 'Relevance': 28}
